In [1]:
import pandas as pd
data = pd.read_csv('pre_processingVer5.csv')
data = data.drop(['Unnamed: 0'], axis = 1)
data.head()

,asin,date,model,product,review,score,user
0,B07S1HTSGH,"June 30, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I am so glad I found this inexpensive camera o...,5.0,Arsenals
1,B07S1HTSGH,"June 30, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,The pictures came out very clear and very real...,5.0,Arsenals
2,B07S1HTSGH,"June 25, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,Now is a good time to travel.I'm a novice phot...,5.0,Vargas Laurence
3,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I'm not sure if it was just the camera that I ...,3.0,Diane
4,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,When I transferred the pictures and videos to ...,3.0,Diane


In [8]:
len(data)

70976

In [6]:
kw = pd.read_csv('KW_list.csv', usecols =[1])
kw_list = list(kw['0'])
kw_str = '|'.join(kw_list)
kw_list

['prosecute',
 'souped up',
 'insufficient',
 'necessitate',
 'major',
 'control',
 'improved',
 'sublimation',
 'high and mighty',
 'arrange',
 'improper',
 'X-rated',
 'creation',
 'hunky dory',
 'demonstration',
 'predominant',
 'solicit',
 'abominable',
 'thirst',
 'lambaste',
 'awful',
 'diabolical',
 'undesirable',
 'outstep',
 'paragon',
 'file a claim',
 'take off',
 'regrettable',
 'stinking',
 'come through',
 'have advantage',
 'finer',
 'baffle',
 'fat',
 'throw',
 'array',
 'dishonest',
 'primary',
 'chief',
 'thirst for',
 'drunken',
 'outgun',
 'weak',
 'yen for',
 'beauty',
 'quell',
 'blameworthy',
 'enhance',
 'drive for',
 'project',
 'main',
 'epitome',
 'self-seeking',
 'preeminent',
 'pomp',
 'go by',
 'excellent',
 'deceitful',
 'pan out',
 'trump',
 'induction',
 'sad',
 'common',
 'break the bank',
 'pattern',
 'wipe off map',
 'puffed up',
 'zero cool',
 'install',
 'lower',
 'improve upon',
 'super',
 'erroneous',
 'hardware',
 'shocking',
 'sinful',
 'recrea

In [4]:
def make_sscs(data):
    from nltk import word_tokenize, pos_tag
    cs_list = []
    ss_list = []
    for i in range(len(data)):
        text = data.loc[i, 'review']
        product = data.loc[i, 'product']
        
        # nltk로 토크나이즈, 포스태깅
        tokens = word_tokenize(text)
        pos_tokens = pos_tag(tokens)
        
        # 비교구문이 들어있는 문장 찾기
        count = 0
        for tok, pos in pos_tokens:
            if pos in ['JJR', 'JJS', 'RBS', "RBR'"]:
                count +=1

        if count != 0:
            cs_list.append(i)
        elif count == 0:
            ss_list.append(i)
    
    df_ss = data.loc[ss_list]     
    
    df_cs = data.loc[cs_list]
    # product_list가 없어서 #처리함. 추후 보강해야 함
#     product_list = []
#     products = '|'.join(product_list)
#     df = df[df.review.str.contains(products)== True]
    df_cs = df_cs[df_cs.review.str.contains(kw_str)== True]
    
    df_cs['CS_SS'] = 'CS'
    df_ss['CS_SS'] = 'SS'

    df_t = df_cs.append(df_ss, ignore_index=True)
    return df_t

In [9]:
df_t = make_sscs(data)

In [10]:
df_t.head()

,asin,date,model,product,review,score,user,CS_SS
0,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I'm not sure if it was just the camera that I ...,3.0,Diane,CS
1,B07S1HTSGH,"July 26, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,"Poor instructions,No clear way to turn off vid...",2.0,Her Babsiness,CS
2,B079GDB7C7,"June 7, 2018",NaN,Canon EOS Rebel T6 DSLR Camera with 18-55mm IS...,I'm deflashed to have a mono-pole but will use...,5.0,michs,CS
3,B079GDB7C7,"October 12, 2018",NaN,Canon EOS Rebel T6 DSLR Camera with 18-55mm IS...,"And you can add better lenses, I have a EF 100...",4.0,Denis Allman,CS
4,B079GDB7C7,"November 26, 2018",NaN,Canon EOS Rebel T6 DSLR Camera with 18-55mm IS...,I couldn't have asked for better packaging wit...,5.0,PrestonPhoto,CS


In [11]:
df_t.to_excel('labelingSSCS.xlsx')